Vamos a hacer un recorrido por los pasos básicos del pre-procesamiento de texto. Estos pasos son necesarios para transformar texto del lenguaje humano a un formato legible para máquinas para su posterior procesamiento.

Veremos cómo realizar estos pasos con código propio, para mayor entendimiento de lo que está sucediendo, y con spaCy.

En concreto, los pasos son:

- Limpieza, la remoción del contenido no deseado.
- Normalización, la conversión diferentes formas a una sola.
- Tokenización, la separación del texto en tókenes (unidades mínimas, por ejemplo palabras).
- Separación en conjuntos de datos: entrenamiento, validación, prueba.
- Generación del vocabulario, la lista de tókenes conocidos.
- Numericalización, el mapeo de tókenes a números enteros.

Nota: El órden de los primeros tres pasos (limpieza, normalización, tokenización) puede variar según conveniencia. El resto de los pasos mantiene el órden.

## Dataset de ejemplo

¿Qué sería de esta publicación sin algunos ejemplos? En nuestro caso vamos a utilizar el dataset CodiEsp. Se trata de un corpus de textos clínicos en español:

In [ ]:
!rm -r /content/sample_data
!wget https://zenodo.org/record/3837305/files/codiesp.zip
!unzip codiesp.zip >> /dev/null

--2023-10-10 22:57:45--  https://zenodo.org/record/3837305/files/codiesp.zip
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11239591 (11M) [application/octet-stream]
Saving to: ‘codiesp.zip’

codiesp.zip         100%[===================>]  10.72M  22.9MB/s    in 0.5s    

2023-10-10 22:57:46 (22.9 MB/s) - ‘codiesp.zip’ saved [11239591/11239591]



In [ ]:
import pandas as pd
from os.path import join

dir_dataset = 'final_dataset_v4_to_publish'
train_dataset = join(dir_dataset, 'train', 'trainX.tsv')

df = pd.read_csv(train_dataset, sep='\t', header=None)

In [ ]:
with pd.option_context('display.max_colwidth', -1):
  display(df.sample(10))

,0,1,2,3,4
1198,S0210-48062005000700015-1,DIAGNOSTICO,m46.90,espondilitis,94 106
2981,S0212-16112012000500043-1,DIAGNOSTICO,f71,retraso mental leve-moderado,725 753
8896,S1699-695X2015000300013-1,DIAGNOSTICO,e04.2,tiroides nódulos,1023 1031;1065 1072
4672,S0376-78922014000200011-1,PROCEDIMIENTO,bl31,RNM muslo derecho,839 842;914 927
7244,S1134-80462015000100006-1,DIAGNOSTICO,j81.1,edema pulmonar,1997 2011
2731,S0212-16112007000800011-1,PROCEDIMIENTO,4a02x4z,electrocardiograma,1524 1542
6588,S1130-14732006000400007-1,PROCEDIMIENTO,b932,RM seno frontal,1795 1797;2275 2287
8539,S1139-76322017000200007-1,DIAGNOSTICO,a23.9,Brucella,1089 1097
8607,S1698-69462006000100006-1,DIAGNOSTICO,r52,doloroso,445 453
2403,S0211-69952014000100016-1,DIAGNOSTICO,n18.9,insuficiencia renal crónica,51 78


In [ ]:
example_text_file = join(dir_dataset,'train','text_files','S0465-546X2014000300010-1.txt')
print(example_text_file)
f = open(example_text_file,'r')
example_text=f.read()
print(example_text)
f.close()

final_dataset_v4_to_publish/train/text_files/S0465-546X2014000300010-1.txt
Mujer, 27 años de edad, profesional de enfermería en área de hospitalización de Medicina Interna, sufre accidente laboral tipo pinchazo con abbocath durante jornada de trabajo el día 23/11/1989. Por lo que se realiza revisión de caso clínico del paciente fuente y las respectivas pruebas confirmatorias, sin encontrarse información en la historia clínica de datos de enfermedades transmisibles. Así mismo las analíticas realizadas en el enfermo y en la trabajadora resultaron negativos los marcadores virales para Hepatitis. 15 días posteriores, presenta alza térmica cuantificada en 40.º, dolor abdominal ubicado en hipocondrio derecho, de leve intensidad, continuo, sin irradiación, no cede con cambios posturales ni con tratamiento ambulatorio, acompañándose de nauseas, vómitos y astenia. Niega coluria, acolia e ictericia, por lo que acude a servicio de urgencia y se decide su ingreso. Durante su estancia se realiza an

##Expresiones Regulares

In [ ]:
import re

##Limpieza
Muchas técnicas modernas no realizan ninguna limpieza alguna. Dependiendo de lo que queremos hacer tal vez convenga deshacernos de algunos elementos.

In [ ]:
def limpiar(texto):
    puntuación = r'[,;.:¡!¿?@#$%&[\](){}<>~=+\-*/|\\_^`"\']'

    # signos de puntuación
    texto = re.sub(puntuación, ' ', texto)

    return texto

limpiar('!!El paciente presenta fiebre/febrícula, tos y mocos y dolor.')

'  El paciente presenta fiebre febrícula  tos y mocos y dolor '

Esta función substituimos los signos de puntuación
, ; . : ¡ ! ¿ ? @ # $ % & [ ] ( ) { } < > ~ = + - * / | \ _ ^ ` " '
por espacios (me gusta más; usar string vacío '' para eliminarlos) mediante expresiones regulares (algunos caracteres tuvieron que ser escapados anteponiendo \ por tener un significado especial para la expresión regular). Hacemos lo mismo con los dígitos. Veamos un ejemplo de funcionamiento.



Otros elementos que podríamos pensar en remover son caracteres invisibles, espacios redundantes. Veremos que esto en particular también puede ser resulto en la tokenización.

## Normalización

Normalizar es la tarea de llevar lo que puede ser expresado de múltiples maneras como fechas, números y abreviaturas a una única forma. Por ejemplo

     13/03/30 -> trece de marzo de dos mil treinta
     DC -> departamento de computación

Se trata de una práctica clásica de la época de los modelos de lenguaje probabilísticos, que intentaban reducir lo más posible la cantidad de palabras. En cierta forma 1 palabra = 1 atributo (lo que en los '90s conocimos como convertibilidad). Elegir atributos en ingeniería de atributos, la parte central del *machine learning*, y lo justamente lo que el *deep learning* busca automatizar.

Sin embargo hay una normalización muy común hoy, el **convertir todo el texto a minúsculas**. En el caso del español, una normalización común es la **remoción de tildes**

In [ ]:
def normalizar(texto):
    # todo a minúsculas
    texto = texto.lower()

    # tildes y diacríticas
    texto = re.sub('á', 'a', texto)
    texto = re.sub('é', 'e', texto)
    texto = re.sub('í', 'i', texto)
    texto = re.sub('ó', 'o', texto)
    texto = re.sub('ú', 'u', texto)
    texto = re.sub('ü', 'u', texto)
    texto = re.sub('ñ', 'n', texto)

    return texto

normalizar('El paciente es español y tomará antibióticos')

'el paciente es espanol y tomara antibioticos'

Hay una librería llamada [unidecode](https://pypi.org/project/Unidecode) que realiza transliteración: representa letras o palabras de un alfabeto en otro, útil si tenemos caracteres en ruso (cirílico) o chino (caracteres Han), aún útil para el alfabeto latino cuando queremos pasar de Unicode a ASCII (lo que substituiría las tildes).


In [ ]:
!pip3 install unidecode --quiet

from unidecode import unidecode

def normalizar_2(texto):

  texto = texto.lower()
  unidecode(texto)

normalizar('El paciente es español y tomará antibióticos')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.9 MB/s eta 0:00:00


'el paciente es espanol y tomara antibioticos'

Una normalización que no vale la pena intentar con este dataset es la **correción ortográfica** con un paquete como [pyspellchecker](https://pypi.org/project/pyspellchecker). Los artículos incluídos en codiesp ya cuentan con corrección ortográfica

#### *Lemmatization*

A diferencia del *stemming*, la lematización reduce las palabras inflexadas a palabras que pertenecen al lenguaje. La raíz pasa a llamarse *lema*.

In [ ]:
!pip install spacy --quiet
!python -m spacy download es_core_news_sm

2023-10-10 23:13:22.448614: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-10 23:13:23.500127: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 54.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


## Tokenización

Tokenizar es separar el texto en partes más pequeñas llamadas tókenes. Una unidad muy común es la palabra pero depende de lo que queramos hacer, si es que no hemos eliminado a los signos de puntuación estos también serían tókenes. Las palabras frecuentemente están compuestas por una raíz, prefijo y/o sufijo, por lo que podríamos decidir separarlos también. En inglés es común separar `it's` en `it` y `'s`, si bien en español esta situación no es común.

A diferencia de la limpieza y la normalización, la tokenización **es un paso indispesable** en la preparación de texto para su procesamiento.

Para el dataset en cuestión la tokenización es simple, vamos a separar seǵun espacios y demás caracteres invisibles como `\t` (tabulación) y `\n` (salto de línea). De haber signos de puntuación, por ejemplo si quisiéramos procesar un documento extenso en oraciones, el proceso es más complejo ya que `final.` tiene un punto en vez de un espacio, y no siempre los puntos demarcan el final de un tóken como en `A.M.` y `P.M.`.

In [ ]:
import spacy

def lematización_tokenizacion(texto):
    nlp = spacy.load('es_core_news_sm')
    doc = nlp(texto)
    doc = [token.lemma_ for token in doc]
    doc = [cadena for cadena in doc if cadena.strip() != ""]
    return doc

resultado = lematización_tokenizacion('Los pacientes son español y tomará antibióticos')
print(resultado)


['el', 'paciente', 'ser', 'español', 'y', 'tomar', 'antibiótico']


Debemos definir si elementos como los signos de puntuación son tókenes o si simplemente delimitan palabras o tókenes, en cuyo caso desaparecerían en el proceso. Mismo con los caracteres invisibles, si estuviésemos haciendo un modelo que programe en Python, la indentación es fundamental y deberiera mantenerse.

`split` también se encarga de los caracteres invisibles repetidos.

Varios modelos de lenguaje utilizan caracteres en vez de palabras como tókenes, esto es útil por varios motivos que listaremos más adelante. Otros utilizan partes de palabras como sílabas (las partes se determinan estadísticamente). Ver https://arxiv.org/pdf/1508.07909.pdf.

### Tokenización utilizando alguna librería

In [ ]:
import spacy

nlp = spacy.load('es_core_news_sm')

doc = nlp('El paciente tiene síntomas de gripe')

print([tóken.text for tóken in doc])

['El', 'paciente', 'tiene', 'síntomas', 'de', 'gripe']


### *Stop words*

Hay listas armadas de palabras muy comunes (*stop words*). Podemos elaborarla de alguna manera o usar alguna existente.

    pip install nltk

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stopwords.words('spanish') [0:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se']

Un detalle a cuidar es que la tokenización usada para la lista de *stop words* tiene que haber sido la misma o similar que la usada para los documentos.

In [ ]:
def filtrar_stop_words(doc):
    return [tóken for tóken in doc if tóken not in stopwords.words('spanish')]

filtrar_stop_words(
    ['el', 'paciente', 'tiene', 'síntomas', 'de', 'gripe'])

['paciente', 'síntomas', 'gripe']

## Otros pre-procesos

Clásicamente se aplicaban alguno de estos para reducir aún más la cantidad de palabras:

#### *Stemming*

*Stem*, de raíz, reduce la inflección de las palabras, mapeando un grupo de palabras a la misma raíz, sin importar si la raíz es una palabras válida en el lenguaje.

     caminando, caminar, camino -> camin

## Primera parte del pre-procesamiento

In [ ]:
def preprocesar(texto):
    texto = limpiar(texto)
    texto = normalizar(texto)
    texto = lematización_tokenizacion(texto)
    texto = filtrar_stop_words(texto)

    return texto

## Conjuntos de datos

En la competencias normalmente encontramos dos archivos, el de entrenamiento y el de inferencia —que le suelen llamar de prueba y es el que tenemos que predecir para entregar—. Del que suelen llamar `train` también tenemos que obtener el de validación.

In [ ]:
train_dataset = join(dir_dataset,'train','trainX.tsv')
test_dataset = join(dir_dataset,'test','testX.tsv')

in_train_df = pd.read_csv(train_dataset, sep = '\t', header=None)
in_test_df = pd.read_csv(test_dataset, sep = '\t', header=None)

in_train_df = in_train_df.loc[in_train_df[1]=='DIAGNOSTICO']
in_test_df = in_test_df.loc[in_test_df[1]=='DIAGNOSTICO']

in_train_df

,0,1,2,3,4
2,S0004-06142005000700014-1,DIAGNOSTICO,n44.8,teste derecho aumentado de tamaño,1343 1376
3,S0004-06142005000700014-1,DIAGNOSTICO,z20.818,exposición a Brucella,594 615
4,S0004-06142005000700014-1,DIAGNOSTICO,r60.9,edemas,1250 1256
5,S0004-06142005000700014-1,DIAGNOSTICO,r52,dolores,78 85
6,S0004-06142005000700014-1,DIAGNOSTICO,a23.9,Brucella,607 615
...,...,...,...,...,...
9176,S2340-98942015000100005-1,DIAGNOSTICO,r06.00,disnea,942 948
9177,S2340-98942015000100005-1,DIAGNOSTICO,c56.2,carcinoma seroso papilar en ovario izquierdo,94 138
9178,S2340-98942015000100005-1,DIAGNOSTICO,r97.1,elevación de CA 125,413 432
9179,S2340-98942015000100005-1,DIAGNOSTICO,r55,pérdida de conocimiento,959 982


In [ ]:
unique_docs = pd.unique(in_train_df[0])
train_df = pd.DataFrame(columns=['text','labels'])
for i in unique_docs:
  doc_df = in_train_df.loc[in_train_df[0]==i]
  labels = []
  for index_, row_ in doc_df.iterrows():
    label = row_[2]
    labels.append(label)
  labels = max(set(labels), key = labels.count)
  train_df = train_df.append({'text':i, 'labels':labels}, ignore_index=True)


unique_docs = pd.unique(in_test_df[0])
test_df = pd.DataFrame(columns=['text','labels'])
for i in unique_docs:
  doc_df = in_test_df.loc[in_test_df[0]==i]
  labels = []
  for index_, row_ in doc_df.iterrows():
    label = row_[2]
    labels.append(label)
  labels = max(set(labels), key = labels.count)
  test_df = test_df.append({'text':i, 'labels':labels}, ignore_index=True)
train_df

<ipython-input-22-f2859bb733f0>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append({'text':i, 'labels':labels}, ignore_index=True)
<ipython-input-22-f2859bb733f0>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append({'text':i, 'labels':labels}, ignore_index=True)
<ipython-input-22-f2859bb733f0>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append({'text':i, 'labels':labels}, ignore_index=True)
<ipython-input-22-f2859bb733f0>:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append({'text':i, 'labels':labels}, ignore_index=True)
<ipython-input-2

,text,labels
0,S0004-06142005000700014-1,a23.9
1,S0004-06142005000900013-1,r58
2,S0004-06142005000900015-1,d49.59
3,S0004-06142005001000011-3,g82.20
4,S0004-06142005001000016-1,n32.89
...,...,...
495,S1889-836X2015000200005-1,r60.9
496,S1889-836X2016000100006-1,m81.0
497,S2254-28842012000300010-1,i82.90
498,S2254-28842014000200009-1,r56.9


Ahora estamos en condiciones de pre-procesar todo lo que tenemos:

In [ ]:
train_docs = []

for doc_name in train_df['text']:
  text_file = join(dir_dataset,'train','text_files','%s.txt' % doc_name)
  f = open(text_file,'r')

  doc_text=f.read()
  plain_text = preprocesar(doc_text)
  f.close()

  train_docs.append(plain_text)

test_docs = []

for doc_name in test_df['text']:
  text_file = join(dir_dataset,'test','text_files','%s.txt' % doc_name)
  f = open(text_file,'r')

  doc_text=f.read()
  plain_text = preprocesar(doc_text)
  f.close()

  test_docs.append(plain_text)

In [ ]:
example_text_file = join(dir_dataset,'train','text_files','S0004-06142005000700014-1.txt')

f = open(example_text_file,'r')
example_text=f.read()
print(example_text)
f.close()

Describimos el caso de un varón de 37 años con vida previa activa que refiere dolores osteoarticulares de localización variable en el último mes y fiebre en la última semana con picos (matutino y vespertino) de 40 C las últimas 24-48 horas, por lo que acude al Servicio de Urgencias. Antes de comenzar el cuadro estuvo en Extremadura en una región endémica de brucella, ingiriendo leche de cabra sin pasteurizar y queso de dicho ganado. Entre los comensales aparecieron varios casos de brucelosis. Durante el ingreso para estudio del síndrome febril con antecedentes epidemiológicos de posible exposición a Brucella presenta un cuadro de orquiepididimitis derecha.
La exploración física revela: Tª 40,2 C; T.A: 109/68 mmHg; Fc: 105 lpm. Se encuentra consciente, orientado, sudoroso, eupneico, con buen estado de nutrición e hidratación. En cabeza y cuello no se palpan adenopatías, ni bocio ni ingurgitación de vena yugular, con pulsos carotídeos simétricos. Auscultación cardíaca rítmica, sin soplos

In [ ]:
train_docs[0]

['describimos',
 'caso',
 'var',
 '37',
 'ano',
 'vida',
 'previo',
 'activo',
 'referir',
 'dolor',
 'osteoarticular',
 'localizacion',
 'variable',
 'ultimo',
 'mes',
 'fiebre',
 'ultimo',
 'semana',
 'pico',
 'matutino',
 'vespertino',
 '40',
 'c',
 'ultima',
 '24',
 '48',
 'hora',
 'acudir',
 'servicio',
 'urgencia',
 'comenzar',
 'cuadro',
 'extremadura',
 'region',
 'endemico',
 'brucella',
 'ingerir',
 'leche',
 'cabra',
 'pasteurizar',
 'queso',
 'dicho',
 'ganado',
 'comensal',
 'aparecer',
 'varios',
 'caso',
 'brucelosis',
 'ingreso',
 'estudio',
 'sindrome',
 'febril',
 'antecedente',
 'epidemiologico',
 'posible',
 'exposicion',
 'brucella',
 'presentar',
 'cuadro',
 'orquiepididimiti',
 'derecho',
 'exploracion',
 'fisico',
 'revelar',
 'tª',
 '40',
 '2',
 'c',
 't',
 '109',
 '68',
 'mmhg',
 'fc',
 '105',
 'lpm',
 'encontrar',
 'consciente',
 'orientado',
 'sudoroso',
 'eupneico',
 'buen',
 'nutricion',
 'hidratacion',
 'cabeza',
 'cuello',
 'palpar',
 'adenopatia',
 'boc

Hemos pasado de una Series de Pandas, array de NumPy o una **lista de strings**

Un poco de nomenclatura: estamos llamando corpus a la colección de textos. Nos referimos también a los textos como documentos. También estamos usando el término lote (*batch*) para referirnos a un (sub)conjunto de documentos.

## Vocabulario

Este paso es importante. Aquí definimos y limitamos la tókenes que vamos a utilizar. El lenguaje es infinito, para convertirlo en un problema tratable muchas veces los que hacemos es reducirlo. Clave para varias prácticas de reducción es contar las frecuencias de los tókenes, esto es, cuántas veces aparece cada tóken en todo el corpus. Como mencionamos las palabras más frecuentes no aportan mucha información y las más infrecuentes si bien son las que más información tienen no llegarán a ser representativas para nuestro modelo. Descartar palabras poco frecuentes también afecta a errores ortográficos.

### Implementación

Veamos cómo acomodamos lo que hemos visto ahora en la clase `Vocab`.

In [ ]:
# versión 4
import numpy as np
from itertools import chain
from collections import Counter

class Vocab():
    @property
    def índice_relleno(self):
        return self.mapeo.get(self.tóken_relleno)

    def __init__(self, tóken_desconocido='<unk>', tóken_relleno='<pad>', frecuencia_mínima=0.0, frecuencia_máxima=0.1,
                 longitud_mínima=1, longitud_máxima=np.inf, stop_words=['paciente', 'síntoma', 'tratamiento'], límite_vocabulario=None):

        self.tóken_desconocido = tóken_desconocido
        self.tóken_relleno = tóken_relleno
        self.frecuencia_mínima = frecuencia_mínima
        self.frecuencia_máxima = frecuencia_máxima
        self.longitud_mínima = longitud_mínima
        self.longitud_máxima = longitud_máxima
        self.stop_words = stop_words
        self.límite_vocabulario = límite_vocabulario

    # ningún cambio aquí
    def reducir_vocabulario(self, lote):
        contador_absoluto = Counter(chain(*lote))

        contador_documentos = Counter()

        for doc in lote:
            contador_documentos.update(set(doc))

        # frecuencia mínima
        if isinstance(self.frecuencia_mínima, int): # frecuencia de tóken
            vocabulario_mín = [tóken for tóken, frecuencia in contador_absoluto.most_common() if frecuencia >= self.frecuencia_mínima]
        else: # frecuencia de documento
            vocabulario_mín = [tóken for tóken, frecuencia in contador_documentos.most_common() if frecuencia/len(lote) >= self.frecuencia_mínima]

        # frecuencia máxima
        if isinstance(self.frecuencia_máxima, int): # frecuencia de tóken
            vocabulario_máx = [tóken for tóken, frecuencia in contador_absoluto.most_common() if self.frecuencia_máxima >= frecuencia]
        else: # frecuencia de documento
            vocabulario_máx = [tóken for tóken, frecuencia in contador_documentos.most_common() if self.frecuencia_máxima >= frecuencia/len(lote)]

        # intersección de vocabulario_mín y vocabulario_máx preservando el órden
        vocabulario = [tóken for tóken in vocabulario_mín if tóken in vocabulario_máx]

        # longitud
        vocabulario = [tóken for tóken in vocabulario if self.longitud_máxima >= len(tóken) >= self.longitud_mínima]

        # stop words
        vocabulario = [tóken for tóken in vocabulario if tóken not in self.stop_words]

        # límite
        vocabulario = vocabulario[:self.límite_vocabulario]

        return vocabulario

    def fit(self, lote):
        vocabulario = self.reducir_vocabulario(lote)

        if self.tóken_desconocido:
            vocabulario.append(self.tóken_desconocido)

        if self.tóken_relleno:
            vocabulario.insert(0, self.tóken_relleno)

        self.mapeo = {tóken: índice for índice, tóken in enumerate(vocabulario)}

        return self

    # ningún cambio aquí
    def transform(self, lote):
        if self.tóken_desconocido: # reemplazar
            return [[tóken if tóken in self.mapeo else self.tóken_desconocido for tóken in doc] for doc in lote]
        else: # ignorar
            return [[tóken for tóken in doc if tóken in self.mapeo] for doc in lote]

    # ningún cambio aquí
    def tókenes_a_índices(self, lote):
        lote = self.transform(lote)

        return [[self.mapeo[tóken] for tóken in doc] for doc in lote]

    # ningún cambio aquí
    def índices_a_tókenes(self, lote):
        mapeo_inverso = list(self.mapeo.keys())

        return [[mapeo_inverso[índice] for índice in doc] for doc in lote]

    def __len__(self):
        return len(self.mapeo)

## El pre-procesamiento hasta ahora

In [ ]:
v = Vocab().fit(train_docs)

train_índices = v.tókenes_a_índices(train_docs)
test_índices = v.tókenes_a_índices(test_docs)

In [ ]:
len(v)

13123

Con esto concluye la primera parte. Hay varias librerías que tienen clases que se encargan de efectuar los pasos que hemos visto. Tienen un comportamiento por defecto, que es configurable (los parámetros que hemos visto) y a su vez, personalizable, para reemplazar algunos o todos los pasos por código propio. En general son librerías desarrolladas por angloparlantes, funcionan *out-of-the-box* bien para el inglés; cuando queremos procesar texto en español vale la pena tener más control sobre estos procesos.

* [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) de scikit-learn.
* [TextDataBunch](https://docs.fast.ai/text.data.html#TextDataBunch.from_df) de fast.ai.

## Pre-procesando las etiquetas
Las etiquetas del dataset también necesitan ser convertidas a números enteros consecutivos. No lo pensamos para este fin pero `Vocab` sería útil en este aspecto. El único tema es que `Vocab.fit` y demás métodos esperan listas de listas de tókenes y a las etiquetas las encontramos en forma de listas de tókenes simplemente.

Podemos llevar la columna de las etiquetas a una lista de listas con `train_df['Intencion'].values.reshape(-1,1)`, de manera de poder interfacearlo con `Vocab`. Algo como `train_df[['Intencion']].values` para que Pandas devuelva un `DataFrame` en vez de una `Series` también funcionaría.

In [ ]:
train_etiquetas = train_df[['labels']].values
test_etiquetas = test_df[['labels']].values

Todo lo que tenga que ver con limitación del vocabulario o agregado de tókenes especiales no nos interesa para este caso de uso.

In [ ]:
vocabulario_etiquetas = Vocab(tóken_desconocido=None).fit(train_etiquetas)

train_etiquetas = vocabulario_etiquetas.tókenes_a_índices(train_etiquetas)
test_etiquetas = vocabulario_etiquetas.tókenes_a_índices(test_etiquetas)

Ya casi estamos. Solo debemos reconvertir a las etiquetas en una lista de índices (su dimensión original) con un recurso que ya conocemos.

In [ ]:
train_etiquetas_índices = list(chain(*train_etiquetas))
test_etiquetas_índices = list(chain(*test_etiquetas))

train_etiquetas_índices[:10]

[68, 16, 37, 69, 70, 5, 71, 72, 5, 38]

Hasta ahora llegamos a convertir esto

```python
[
    'que se requiere para un prestamo personal',
    'me piden mi numero de cuenta es mi cbu',
]
```

en esto

```python
[
    [4160, 4683, 4484, 3703, 5294, 4011, 3825],
    [3275, 3854, 3319, 3554, 1532, 1462, 2151, 3319, 950],
]
```

donde dijimos que las partes fundamentales son la tokenización —separar a los documentos en unidades de información— y la numericalización —el asignarle a cada uno de los tókenes un número, más que nada para que la computadora, que gusta mucho de los números, sea feliz—.

También habíamos dicho que **un tóken es un atributo** pero no dijimos mucho más al respecto. Veamos cómo puede ser esto. La tarea de ejemplo es clasificar documentos. Estamos acustumbrados a tener muestras y etiquetas como `X` e `y` en las que la primera es una matriz de muestras (filas) y atributos (columnas), y la segunda suele ser una columna. Cuando el dataset está sin pre-procesar tenemos las muestras (filas) pero no los atributos (columnas), por lo general tenemos una única columna con los documentos en forma de strings, lo que mucha forma de atributos no tiene.

Ahora que hemos pre-procesado el texto estamos a un paso de obtener los atributos. La función de los atributos es describir o caracterizar a las muestras. El modelo lee estos atributos para realizar inferencias. Hay distintas maneras de describir a los documentos, algunas más sofisticadas que otras, una intuitiva es aprovechar que los tókenes están numerados desde 0 hasta L (`len(vocabulario)`) y otorgarle una columna a cada uno en la matriz de atributos de tamaño N x L (donde N es la cantidad de muestras).

Hecho esto, solo resta contar cuántas veces aparece cada tóken en cada documento y asentarlo en la matriz.

```
                  |  bien  hola  si    todo
-------------------------------------------
'hola todo bien'  |  1     1     0     1
'si bien bien'    |  2     0     1     0
```

Como comentario, esta forma de describir los documentos ignora enteramente el órden de los tókenes, sabemos que el sentido de una oración puede cambir completamente si cambiamos algunas palabras de lugar. Para el problema en cuestión, no parece ser tan grave ya que para clasificar una pregunta podría bastar con reconocer algunas palabras claves como *cambio* y *clave* o *requisito* y *préstamo*.

Ver [tf-idf](https://es.wikipedia.org/wiki/Tf-idf).

## PyTorch

El típico bucle de entrenamiento de PyTorch tiene esta pinta.

```python
for época in range(N_ÉPOCAS):
    for lote in datos_entrenamiento:
        # reseteamos los gradientes
        optimizador.zero_grad()
        
        predicciones = red_neuronal(lote.X)
        pérdida = criterio(predicciones, lote.y)
        
        # calculamos los gradientes
        pérdida.backward()
    
        # aplicamos los gradientes
        optimizador.step()
```

Recordemos que a diferencia de otros modelos las redes neuronales revisitan varias veces el dataset, en lo que se llaman épocas, cada época es un recorrido por todas las muestras de entrenamiento.

En una época el dataset se puede mostrar entero, de a una muestra, o como es común hoy en día de a grupos o lotes (*batches*). La experiencia mostró que es útil variar el orden de las muestras en cada época.

## PyTorch

El típico bucle de entrenamiento de PyTorch tiene esta pinta.

```python
for época in range(N_ÉPOCAS):
    for lote in datos_entrenamiento:
        # reseteamos los gradientes
        optimizador.zero_grad()
        
        predicciones = red_neuronal(lote.X)
        pérdida = criterio(predicciones, lote.y)
        
        # calculamos los gradientes
        pérdida.backward()
    
        # aplicamos los gradientes
        optimizador.step()
```

Recordemos que a diferencia de otros modelos las redes neuronales revisitan varias veces el dataset, en lo que se llaman épocas, cada época es un recorrido por todas las muestras de entrenamiento.

En una época el dataset se puede mostrar entero, de a una muestra, o como es común hoy en día de a grupos o lotes (*batches*). La experiencia mostró que es útil variar el orden de las muestras en cada época.



PyTorch provee ciertas facilidades para el manejo de los datos con las clases definidas en [torch.utils.data](https://pytorch.org/docs/stable/data.html) a ser:
1. `Dataset`. Organiza los datos. Le pasamos un número o índice de muestra y nos devuelve la muestra usualmente como una tupla `(atributos, etiqueta)`.
1. `Sampler`. Salvo que lo queramos de otra manera, se encarga de brindar un orden aleatoreo de los índices del dataset; uno diferente cada vez que le preguntamos.
1. `BatchSampler`. Por defecto, se inicializa con un `Sampler` y el tamaño de lote. Se encarga de armar grupos de índices; diferentes cada vez que le preguntamos.
1. `DataLoader`. Valiéndose de los grupos de índices de `BatchSampler`, obtiene muestras de `Dataset`. De esta manera para cada época devuelve lotes de muestras al azar.  

Por `Sampler` y `BatchSampler` no nos detendremos ya el comportamiento por defecto, que es barajar el dataset en cada época y armar lotes del mismo tamaño es todo lo que necesitamos.

## DataLoader

`DataLoader` es un *iterable*. Los iterables son colecciones de elementos que se pueden recorrer; implementan el método `__iter__`, del que se espera que devuelva un objeto *iterador* (`iterador = iter(iterable)`). A su vez el iterador implementa el método `__next__` que se encarga devolver secuencialmente los elementos de la colección hasta que se agota; una vez que esto sucede el iterador debe ser descartado y en todo caso le pedimos al iterable que nos arme un nuevo iterador. Cuando usamos la construcción `for ítem in iterable`, el intérprete de Python implícitamente obtiene un iterador.

Ver la [sección de interables](https://docs.python.org/3/tutorial/classes.html#iterators) en el tutorial de Python.

In [ ]:
lista = iter(['uno','dos'])

next(lista)

'uno'

In [ ]:
from torch.utils.data import DataLoader

train_dl = DataLoader(train_ds, batch_size=20, shuffle=True)

*Le* estamos diciendo a `DataLoader` que queremos lotes de 32 muestras (`batch_size`) y que el armado de los lotes sea aleatorio (`shuffle`).

In [ ]:
def rellenar_documentos(lote, largos, índice_relleno):
    máximo_largo = max(largos)

    return [doc + [índice_relleno] * (máximo_largo - largos[i]) for i, doc in enumerate(lote)]

In [ ]:
class AtriDicc():
    def __init__(self, *args, **kwargs):
        self.__dict__ = dict(*args, **kwargs)

    def __repr__(self):
        return repr(self.__dict__)

In [ ]:
from torch.utils.data import Dataset

class Textset(Dataset):
    def __init__(self, documentos, etiquetas=None):

        self.documentos = documentos
        self.etiquetas  = etiquetas or np.full(len(documentos), np.nan)

    def __len__(self):
        return len(self.documentos)

    def __getitem__(self, item):
        return AtriDicc(
            documento = self.documentos[item],
            largo = len(self.documentos[item]),
            etiqueta =  self.etiquetas[item],
        )

In [ ]:
train_ds = Textset(train_índices, train_etiquetas_índices)
len(train_ds)
train_ds[400].documento[:10]

[13122, 6107, 13122, 13122, 13122, 13122, 1813, 6075, 13122, 13122]

## Función *collate*

*Collate* significa juntar diferentes piezas de información para ver sus similaridades y diferencias, también puede ser colectar y organizar las hojas de un reporte, un libro. En el contexto de `DataLoader` quiere decir arreglar el lote. Entonces esta función recibe una lista de elementos del `Dataset`, en nuestro caso una lista de de `AtriDicc`s, y debe devolver el lote en una forma útil y en lo posible realizar conversiones a tensores.

`DataLoader` posee una *collate function* por defecto que utiliza internamente y que en muchos casos funciona correctamente, pero otros como ahora que tenemos documentos de distinto largo nos toca definir una función propia.

In [ ]:
def rellenar_lote(lote):
    """Prepara lotes para ingresar a nn.Embedding"""
    documentos = [elemento.documento for elemento in lote]
    largos     = [elemento.largo     for elemento in lote]
    etiquetas  = [elemento.etiqueta  for elemento in lote]

    rellenos = rellenar_documentos(documentos, largos, v.índice_relleno)

    return AtriDicc(
        documentos = torch.tensor(rellenos),
        etiquetas  = torch.tensor(etiquetas),
    )

Cuando instanciamos un `DataLoader` le pasamos la función que acabamos de definir.

In [ ]:
train_dl = DataLoader(train_ds, collate_fn=rellenar_lote, batch_size=3, shuffle=True)

In [ ]:
import torch

un_lote = next(iter(train_dl))
un_lote.documentos

tensor([[13122,   547, 13122,  ..., 13122, 13122,  4453],
        [13122,   390, 13122,  ...,     0,     0,     0],
        [  217, 13122, 13122,  ...,     0,     0,     0]])

In [ ]:
un_lote.etiquetas

tensor([49, 17, 26])

## Fuentes consultadas

https://github.com/matiasbattocchia/datitos/blob/master/_notebooks/2020-07-23-Preprocesamiento-de-texto-para-NLP-parte-1.ipynb
https://github.com/matiasbattocchia/datitos/blob/master/_notebooks/2020-07-30-Preprocesamiento-de-texto-para-NLP-parte-2.ipynb